In [2]:
import os
import json
import glob
import gzip
import shutil
from tqdm import tqdm
import pickle

import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from deepl import DeepLCLI
import googletrans

## Unzip JSON Files

In [5]:
# unzip_data_path = glob.glob("../data/c4/realnewslike/*.gz")
# save_dir = "./data/c4"
# for gz_file in unzip_data_path:
#     with gzip.open(gz_file, 'rb') as f_in:
#         file_name = (".").join(gz_file.split("/")[-1].split(".")[:-1])
#         with open(os.path.join(save_dir,file_name), 'wb') as f_out:
#             shutil.copyfileobj(f_in, f_out)

In [6]:
# os.path.join(save_dir,file_name)

## Merging Files

In [7]:
# train_files = glob.glob("data/c4_realnewslike/*train*.json")
# valid_files = glob.glob("data/c4_realnewslike/*valid*.json")
# len(train_files), len(valid_files)

In [8]:
# val_df = pd.read_json(valid_files[0], lines=True)

# train_df = pd.DataFrame()
# for file in tqdm(train_files, desc="merge JSON ..."):
#     tmp = pd.read_json(file, lines=True)
#     train_df = pd.concat([train_df, tmp], axis=0)

In [9]:
# print(train_df.shape, val_df.shape)

## Save to CSV

In [10]:
# with open("data/train_total_realnewslike.pkl", "wb") as f:
#     pickle.dump(train_df, f)
#     f.close()

# with open("data/valid_total_realnewslike.pkl", "wb") as f:
#     pickle.dump(val_df, f)
#     f.close()

In [11]:
# train_df.head()

## Translate with DeepL

In [12]:
with open("data/train_total_realnewslike.pkl", "rb") as f:
    train_df = pickle.load(f)
    f.close()

with open("data/valid_total_realnewslike.pkl", "rb") as f:
    valid_df = pickle.load(f)
    f.close()

train_df.head()

,text,timestamp,url
0,Spain's anti-austerity Podemos movement has vo...,2019-04-19 13:47:12+00:00,https://www.bbc.com/news/world-europe-32874790
1,Above: Gosling and Stone make an enjoyable cou...,2019-04-19 00:45:34+00:00,https://www.businesstimes.com.sg/lifestyle/mov...
2,QUEBEC — The stormy debate over the arrest of ...,2019-04-20 23:02:05+00:00,https://nationalpost.com/pmn/news-pmn/canada-n...
3,Dunkirk was filmed at the French location of t...,2019-04-19 15:30:54+00:00,https://deadline.com/2018/04/dunkirk-baby-driv...
4,Zimbabwe's President Emmerson Mnangagwa has la...,2019-04-18 19:02:42+00:00,http://www.businessghana.com/site/news/Politic...


## DeepL CLI Configuration

In [13]:
deepl = DeepLCLI("en", "ko")
# deepl.timeout = 15000
# deepl.max_length = 3000

## Whole Dataset

In [14]:
def deepl_translate(df):
    deepl = DeepLCLI("en", "ko")
    kor = []
    for idx in tqdm(range(len(df)), desc="Translate ... "):
        if 3000<len(df.iloc[idx,0])<=6000:
            tmp = df.iloc[idx,0].split("\n")

            front = (" ").join(tmp[:len(tmp)//2])
            back = (" ").join(tmp[len(tmp)//2:])

            front_tr = deepl.translate(front)
            back_tr = deepl.translate(back)

            kor.append((front_tr+back_tr).replace("\n", " "))

        elif len(df.iloc[idx,0])>6000:
            continue
        else:
            tr = deepl.translate(df.iloc[idx,0])

            kor.append(tr.replace("\n", " "))
    return kor

In [22]:
def google_translate(df):
    translator = googletrans.Translator()
    kor = []
    
    for idx in tqdm(range(len(df)), desc="Translate ... "):
        kor.append((translator.translate(df.iloc[idx,0], src="en", dest="ko").text).replace("\n", " "))

In [24]:
google_translate(valid_df)

Translate ... :   0%|▏                                                                     | 37/13863 [00:13<1:25:44,  2.69it/s]


KeyboardInterrupt: 

In [ ]:
valid_df["kor"] = translate(valid_df)
train_df["kor"] = translate(train_df)

In [ ]:
front_tr = deepl.translate_async(front)

In [4]:
import deepl
deepl.__version__

'0.5.0'